# Configuração do ambiente

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from typing import Dict
from time import time

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Obtenção dos Dados

In [88]:
size = 100
# Dados de entrada sobre as frutas para a geração dos dados aleatórios.
inputs = [{'Class':     'Banana', 'Mean_height': 18, 'Std_height':   3, 'Mean_width':  5, 'Std_width':   1},
          {'Class':      'Apple', 'Mean_height': 10, 'Std_height': 1.5, 'Mean_width': 12, 'Std_width': 1.5},
          {'Class':      'Lemon', 'Mean_height':  5, 'Std_height':   1, 'Mean_width':  5, 'Std_width':   1},
          {'Class': 'Watermelon', 'Mean_height': 20, 'Std_height': 2.5, 'Mean_width': 25, 'Std_width':   3},
          {'Class':     'Papaya', 'Mean_height': 28, 'Std_height':   3, 'Mean_width': 15, 'Std_width':   2},
          {'Class':      'Mango', 'Mean_height': 17, 'Std_height': 1.5, 'Mean_width': 12, 'Std_width': 1.5},
          {'Class': 'Strawberry', 'Mean_height':  4, 'Std_height': 0.5, 'Mean_width':  2, 'Std_width': 0.5},
          {'Class':      'Grape', 'Mean_height':  2, 'Std_height': 0.5, 'Mean_width':  2, 'Std_width': 0.5}]

heights = np.array([])
widths = np.array([])
classes = []
# Para cada entrada, acessa a média e o desvio padrão da altura e largura de cada
# fruta e gera uma amostra com base da distribuição normal desses dados.
for input_i in inputs:
  loc_h   = input_i['Mean_height']
  scale_h = input_i['Std_height']
  height = np.random.normal(loc=loc_h, scale=scale_h, size=size)
  heights = np.concatenate([heights, height])

  loc_w   = input_i['Mean_width']
  scale_w = input_i['Std_width']
  width  = np.random.normal(loc=loc_w, scale=scale_w, size=size)
  widths = np.concatenate([widths, width])

  class_i = {
      'Banana': 0,
      'Apple': 1,
      'Lemon': 2,
      'Watermelon': 3,
      'Papaya': 4,
      'Mango': 5,
      'Strawberry': 5,
      'Grape': 6
  }[input_i['Class']]

  classes += [class_i] * size

classes = np.array(classes)

# Junta os dois arrays e transpôe
data = np.vstack((heights, widths)).T
data

array([[15.37695708,  7.09735538],
       [17.76967499,  6.02249663],
       [20.30115499,  4.42510384],
       ...,
       [ 2.938452  ,  2.62362593],
       [ 3.48533781,  1.60582745],
       [ 2.76358718,  1.73173853]])

# Teste com SKlearn

In [65]:
st = time()

X = data
y = classes

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o classificador de árvore de decisão
clf = DecisionTreeClassifier()

# Treinar o classificador com os dados de treinamento
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Calcular a precisão das previsões
accuracy = accuracy_score(y_test, y_pred)

print(f'A precisão do classificador de árvore de decisão é: {accuracy}')

print(f'Tempo de execução: {time()-st} Segundos')

A precisão do classificador de árvore de decisão é: 0.99375
Tempo de execução: 0.04439854621887207 Segundos


# Implementação

In [192]:
class Node:
  def __init__(self, mask: np.array):
    self.mask = mask
    self.leaves = list()

  def add_leaf(self, mask):
    new_mask = [m_i & m_j for m_i, m_j in zip(self.mask, mask)]
    new_node = Node(new_mask)
    self.leaves.append(new_node)
    return new_node

In [200]:
class Tree:
  def __init__(self, data: np.array, classes: np.array, indice: str = 'Gini'):
    self.data = data
    self.classes = classes
    # Inicializa a raiz com uma máscara que é toda True.
    self.root = Node(np.full(data.shape[0], True))

    self.fit(self.root)


  def fit(self, node: Node, n_intervals: int = 11, limit: float = 0.2):

    # Acessa as restrições do ramo
    mask = node.mask

    n_features = self.data.shape[1]

    min_impureza = 1
    best_feature = 0
    best_interval = 0
    for feature in range(n_features):
      # Pega o min e o max de cada coluna e divide em n intervalos
      x_min = np.min(data[:,feature])
      x_max = np.max(data[:,feature])
      step = (x_max-x_min)/n_intervals
      intervals = np.arange(x_min+step, x_max, step)
      # Calcula a impureza de cada intervalo
      for x in intervals:
        '''
        maski: uma máscara booleana que analisa se a coluna i é <= ou > que x.
        pi: partição dos dados que atendem à máscara booleana.
        ci: partição das classes que atendem à máscara booleana.
        Ao sobrescrever ci por np.bincount(c1), é realizada a contagem de cada classe.
        gi: a impureza da partição i.
        '''
        # -> Primeira partição
        mask1 = self.data[:, feature] <= x
        # Junta a nova máscara com a máscara princial
        mask1 = [m_i & m_j for m_i, m_j in zip(mask, mask1)]
        p1 = self.data[:, feature][mask1]
        c1 = self.classes[mask1]
        c1 = np.bincount(c1)
        s1 = sum(c1) # Total de elementos em p1
        g1 = 1 - sum([(i/s1)**2 for i in c1])
        # -> Segunda partição
        mask2 = self.data[:, feature]  > x
        # Junta a nova máscara com a máscara princial
        mask2 = [m_i & m_j for m_i, m_j in zip(mask, mask2)]
        p2 = self.data[:, feature][mask2]
        c2 = self.classes[mask2]
        c2 = np.bincount(c2)
        s2 = sum(c2) # Total de elementos em p2
        g2 = 1 - sum([(i/s2)**2 for i in c2])
        # Total de elementos
        total = s1 + s2

        impureza_media = (g1*s1 + g2*s2) / total
        if impureza_media < min_impureza:
          min_impureza = impureza_media
          best_feature = feature
          best_interval = x

    node1 = node.add_leaf(mask1)
    if g1 > limit:
      self.fit(node1)
    node2 = node.add_leaf(mask2)
    if g2 > limit:
      self.fit(node2)
    # Cria os dois novos nós
    return min_impureza, best_feature

In [202]:
x = Tree(data, classes)